In [1]:
import pandas as pd
import numpy as np
import stats
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
import statistics 
import time
import itertools

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, LSTM, ConvLSTM2D, Dropout,BatchNormalization, Bidirectional, Conv1D, MaxPooling1D, TimeDistributed, Flatten
from tensorflow.keras.utils import plot_model

from scipy import stats
from sklearn.preprocessing import normalize, RobustScaler, OneHotEncoder

In [2]:
class Single_Dataset:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.number_of_instances = len(y)
        
    def export_new_batch(self, no_of_rounds):
        round_batch_size = -(-self.number_of_instances // no_of_rounds)
        indices = np.arange(self.X.shape[0])
        np.random.shuffle(indices)
        
        self.X = self.X[indices]
        self.y = self.y[indices]
        
        X_batch = self.X[0:round_batch_size]
        self.X = np.delete(self.X,  np.s_[0:round_batch_size], axis = 0)

        y_batch = self.y[0:round_batch_size]
        self.y = np.delete(self.y,  np.s_[0:round_batch_size], axis = 0)
        
        return(X_batch, y_batch)
        
class Data:
    def __init__(self, path):
        all_files = glob.glob(path + "/*.txt")

        li = []

        for filename in all_files:
            df = pd.read_csv(filename, index_col=None, header=None)
            li.append(df)

        self.data = pd.concat(li, axis=0, ignore_index=True)
        self.data.columns = ['User', 'Activity', 'timestamp', 'X', 'Y', 'Z']
        self.data.Z.replace(regex=True, inplace=True, to_replace=r';', value=r'')
        self.data['Z'] = self.data.Z.astype(np.float64)
        self.data.dropna(axis=0, how='any', inplace=True)
        self.train_data = []
        self.test_data = []
        self.edge_sets = []
        self.central_set = []

    def include_activities(self, activities):
        if len(activities) > 0:
            self.data = self.data[self.data['Activity'].isin(activities)]
            
    def train_test_split(self, users_in_train):
        train_users = self.data.User.unique()[:users_in_train]
        test_users = self.data.User.unique()[users_in_train:]
        self.train_data = self.data[self.data['User'].isin(train_users)]
        self.test_data = self.data[self.data['User'].isin(test_users)]
    
    def split_to_edge_sets(self, window_size, stride, features): #[users,[activities]]
        train_users = self.data.User.unique()[:40]
        set_args = [[train_users[:20], ['A', 'B', 'C']], 
                    [train_users[20:], ['A', 'B', 'D']]]
        for arg in set_args:
            dataset = self.data[(self.data['Activity'].isin(arg[1])) & (self.data['User'].isin(arg[0]))]
            dataset = dataset.drop(['User', 'timestamp'], axis = 1)
            X,y = self.format_(dataset, window_size, stride, features)
            edge_set = Single_Dataset(X,y)
            self.edge_sets.append(edge_set)
            
    def format_(self, data, window_size, stride, features):
        encoder = OneHotEncoder(handle_unknown='ignore', sparse=False).fit(self.data.Activity.values.reshape(-1, 1))
        X = np.array(data[['X', 'Y', 'Z']])
        Xs, ys = [], []
        for i in range(0, len(X) - window_size, stride):
            labels = data['Activity'].iloc[i: i + window_size]
            try:
                modelabel = stats.mode(labels)[0][0]
                v = X[i:(i + window_size)]
                Xs.append(v)
                ys.append(modelabel)
            except: continue
        Xs, ys = np.array(Xs), np.array(ys)
        ys = encoder.transform(ys.reshape(-1, 1))
        return Xs, ys
        
    def format_test_set(self, window_size, stride, features):
        Xs, ys = self.format_(self.test_data, window_size, stride, features)
        self.central_set.append(Single_Dataset(Xs, ys))
        
class Node:
    def __init__(self, model):
        self.model = model
        self.model_weights = model.get_weights()
        self.local_gradients = np.zeros(shape=np.shape(model.get_weights()))
        self.X = []
        self.y = []
    
    def update_model(self, gradient_bin):
        weights = self.model.get_weights()
        for update in gradient_bin:
            index = update[0]
            gradient = update[1]
            l = len(index)
            if l == 1: weights[index] += gradient
            elif l == 2: weights[index[0]][index[1]] += gradient
            elif l == 3: weights[index[0]][index[1]][index[2]] += gradient
            elif l == 4: weights[index[0]][index[1]][index[2]][index[3]] += gradient
            elif l == 5: weights[index[0]][index[1]][index[2]][index[3]][index[4]] += gradient
        self.model.set_weights(weights)
        
    def update_local_gradients(self):
                gradients = np.array(self.model.get_weights()) - np.array(self.model_weights)
                self.local_gradients = np.add(self.local_gradients, gradients)
                self.model_weights = self.model.get_weights()
        
    def pop_top_k_gradients(self, percentage):
        gradient_bin = []
        for idx_1, set_1 in enumerate(self.local_gradients):
            if isinstance(set_1, (np.ndarray, list)):
                for idx_2, set_2 in enumerate(set_1):
                    if isinstance(set_2, (np.ndarray, list)):
                        for idx_3, set_3 in enumerate(set_2):
                            if isinstance(set_3, (np.ndarray, list)):
                                for idx_4, set_4 in enumerate(set_3):
                                    gradient_bin.append([[idx_1, idx_2, idx_3, idx_4], set_4])
                            else: gradient_bin.append([[idx_1, idx_2, idx_3], set_3])  
                    else: gradient_bin.append([[idx_1, idx_2], set_2])  
            else: gradient_bin.append([[idx_1], set_1])  

        gradient_bin = sorted(gradient_bin,key=lambda g:abs(g[1]), reverse=True)
        gradient_bin = gradient_bin[: int(len(gradient_bin) * percentage)]
        
        for update in gradient_bin:
            index = update[0]
            gradient = update[1]
            l = len(index)
            if l == 1: self.local_gradients[index] = 0
            elif l == 2: self.local_gradients[index[0]][index[1]] = 0
            elif l == 3: self.local_gradients[index[0]][index[1]][index[2]] = 0
            elif l == 4: self.local_gradients[index[0]][index[1]][index[2]][index[3]] = 0
            elif l == 5: self.local_gradients[index[0]][index[1]][index[2]][index[3]][index[4]] = 0
                
        return gradient_bin
   
class Edge_node(Node):
    def __init__(self, model):
        Node.__init__(self, model)
        
    def train_model(self):
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15, restore_best_weights=True)

        history = self.model.fit(
            self.X, self.y,
            epochs=500,
            batch_size=32,
            validation_split=0.3,
            shuffle=True,
            callbacks = es,
            verbose = False
        )
            
class Central_Node(Node):
    def __init__(self, model):
        Node.__init__(self, model)
        
    def evaluate_model(self):
        predictions = self.model.predict(self.X)
        loss, acc = self.model.evaluate(self.X, self.y, verbose=0)
        y_pred = tf.argmax(predictions, axis=-1)
        y_true = tf.argmax(self.y, axis=-1)
        cm = tf.math.confusion_matrix(tf.reshape(y_true, [-1]),
                                 tf.reshape(y_pred, [-1]))
        diagonal = tf.linalg.diag_part(cm)
        recalls = diagonal / tf.reduce_sum(cm, axis=1)
        return acc, loss, cm, recalls
    
class Gradient_Accumulator:
    def __init__(self):
        self.gradient_list = []

    def store_gradients(self, gradients):
        self.gradient_list.extend(gradients)

    def return_avg_gradients(self):
        merged_gradients = []
        self.gradient_list.sort(key = lambda x: x[0])
        for key, group in itertools.groupby(self.gradient_list, lambda x : x[0]):
            values = [x[1] for x in group]
            merged_gradients.append([key, sum(values)/len(values)])
        return merged_gradients

    def return_max_gradients(self):
        merged_gradients = []
        self.gradient_list.sort(key = lambda x: x[0])
        for key, group in itertools.groupby(self.gradient_list, lambda x : x[0]):
            values = [x[1] for x in group]
            merged_gradients.append([key, max(values)])
        return merged_gradients

    def empty_gradient_list(self):
        self.gradient_list = []

In [3]:
def define_model(n_timesteps, n_features, categories_number):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(categories_number, activation='softmax'))

    model.compile(
      loss='categorical_crossentropy',
      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
      metrics=['acc']
    )
    return model

def clone_model(origin_model):
    model_copy= keras.models.clone_model(origin_model)
    model_copy.build() # replace 10 with number of variables in input layer
    
    model_copy.compile(
      loss='categorical_crossentropy',
      optimizer='adam',
      metrics=['acc']
    )
    model_copy.set_weights(origin_model.get_weights())
    return model_copy

def transfer_data_set_to_node(edge_set, edge_node, rounds_number):
    X,y = edge_set.export_new_batch(rounds_number)
    edge_node.X = X
    edge_node.y = y
    
def transfer_gradients(node_from, node_to, top_k_gradients_percentage):
    gradients_to_sent = node_from.pop_top_k_gradients(top_k_gradients_percentage)
    node_to.update_model(gradients_to_sent)

In [4]:
window_size = 100 
stride = 20
steps = 4 
dimentions = 1 
sub_window_size = 25 
features = 3
categories = 4

dataset = Data('wisdm-dataset/raw/watch/accel')
dataset.include_activities(['A', 'B', 'C', 'D'])
dataset.train_test_split(40)
dataset.split_to_edge_sets(window_size, stride, features)
dataset.format_test_set(window_size, stride, features)

In [5]:
conv_model = define_model(window_size, features, categories)
edge_list = []

model = clone_model(conv_model)

central_node = Central_Node(model)
central_node.X = dataset.central_set[0].X
central_node.y = dataset.central_set[0].y

gradient_accumulator = Gradient_Accumulator()

for edge_index in range(2):
    model = clone_model(conv_model)
    e = Edge_node(model)
    edge_list.append(e)

In [6]:
score_bin = []
rounds_number = 10
top_k_gradients_percentage = 100
for round_n in range(rounds_number):
    for edge_set, edge_node in zip(dataset.edge_sets, edge_list):
        transfer_data_set_to_node(edge_set, edge_node, rounds_number)
        if round_n > 0:
            transfer_gradients(central_node, edge_node, top_k_gradients_percentage)
        edge_node.train_model()
        edge_node.update_local_gradients()
        gradients_to_sent = edge_node.pop_top_k_gradients(top_k_gradients_percentage)
        gradient_accumulator.store_gradients(gradients_to_sent)
    merged_gradients = gradient_accumulator.return_avg_gradients()
    gradient_accumulator.empty_gradient_list()
    central_node.update_model(merged_gradients)
    acc, loss, cm, recalls = central_node.evaluate_model()
    score_bin.append([top_k_gradients_percentage, rounds_number, round_n, acc, loss, cm, recalls])
    central_node.update_local_gradients()

Restoring model weights from the end of the best epoch.
Epoch 00030: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00022: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00019: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00019: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00041: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00019: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00024: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00022: early s

In [7]:
score_bin

[[100,
  10,
  0,
  0.4711498022079468,
  1.7090519666671753,
  <tf.Tensor: shape=(4, 4), dtype=int32, numpy=
  array([[1977,  448,    0,    0],
         [   2, 2244,    0,    0],
         [2300,  131,    1,    0],
         [1996,  119,   45,  269]], dtype=int32)>,
  <tf.Tensor: shape=(4,), dtype=float64, numpy=array([8.15257732e-01, 9.99109528e-01, 4.11184211e-04, 1.10745163e-01])>],
 [100,
  10,
  1,
  0.4744020104408264,
  1.3373231887817383,
  <tf.Tensor: shape=(4, 4), dtype=int32, numpy=
  array([[2078,  335,   12,    0],
         [   1, 2245,    0,    0],
         [2244,  148,   34,    6],
         [1015,    5, 1244,  165]], dtype=int32)>,
  <tf.Tensor: shape=(4,), dtype=float64, numpy=array([0.85690722, 0.99955476, 0.01398026, 0.06792919])>],
 [100,
  10,
  2,
  0.5784724950790405,
  1.6831034421920776,
  <tf.Tensor: shape=(4, 4), dtype=int32, numpy=
  array([[2209,  211,    2,    3],
         [   2, 2243,    1,    0],
         [2328,   99,    3,    2],
         [1279,    7,   8